In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {'scroll': True,})

%load_ext autoreload
%autoreload 2

# Robot model

# Kinematic model

A robotic arm is considered as a kinematic chain of rigid bodies (or links) connected by joints to provide constrained motion.
These links are constrained by their connections to other links.

All rigid body movements are rotations, translations, or combinations of the two.

kinematics = subdomain of mechanics, contrary to dynamics it concerns laws of motion without considering forces

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/01_robot_links_and_joints.jpg" />

## Kinematic chains

<table>
<tr>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/2/2c/Modele_cinematique_corps_humain.svg" style="height: 300px" /></td>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/ATHLETE_robot_climbing_a_hill.jpg/640px-ATHLETE_robot_climbing_a_hill.jpg" style="height: 300px" /></td>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/f/f2/SteamEngine_Boulton%26Watt_1784.png" style="height: 300px" /></td>
</tr>
</table>

In the kinematic model of a robot, the connection of different manipulator joints is known as link, and the 
integration of two or more links is called a joint. This kinematic model can be represented as a tree structure.
The tree describes the kinematic chain, i.e., the connection of robotic links with joints, and the
inter-dependendencies of these links.
This tree structure plus the underlying geometric information can be defined in Unified Robot Description Format (URDF),
which describes any robot. If the robot is mounted on external axes, these links and joints
can be added as well.

In [ ]:
TODO:

open kinematic chain: there is only one sequence of links connecting the two ends on the chain
    Closed kinematics: when a sequence of links forms a loop

## Unified Robot Description Format (URDF)

### Part of ur5.urdf

```xml
<link name="base_link">
    <visual>
      <geometry>
        <mesh filename="package://ur_description/meshes/ur5/visual/base.obj"/>
      </geometry>
      <material name="LightGrey">
        <color rgba="0.7 0.7 0.7 1.0"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <mesh filename="package://ur_description/meshes/ur5/collision/base.stl"/>
      </geometry>
    </collision>
    <inertial>
      <mass value="4.0"/>
      <origin rpy="0 0 0" xyz="0.0 0.0 0.0"/>
      <inertia ixx="0.00443333156" ixy="0.0" ixz="0.0" iyy="0.00443333156" iyz="0.0" izz="0.0072"/>
    </inertial>
  </link>
  
  <joint name="shoulder_pan_joint" type="revolute">
    <parent link="base_link"/>
    <child link="shoulder_link"/>
    <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.089159"/>
    <axis xyz="0 0 1"/>
    <limit effort="150.0" lower="-6.2831853" upper="6.2831853" velocity="3.15"/>
    <dynamics damping="0.0" friction="0.0"/>
  </joint>
```

### Example UR5 URDF:

https://github.com/ros-industrial/universal_robot/blob/kinetic-devel/ur_description/urdf/ur5.urdf.xacro

In [1]:
"""Example: Create robot from URDF.
"""
import compas_fab
from compas.robots import RobotModel
from compas.robots import GithubPackageMeshLoader

from viewer import RobotArtist
from viewer import ThreeJsViewer

# repo, package, branch
r = 'ros-industrial/abb'
p = 'abb_irb6600_support'
b = 'kinetic-devel'

# download urdf and meshes from github
loader = GithubPackageMeshLoader(r, p, b)
urdf = loader.load_urdf('irb6640.urdf')

# create robot model from URDF
model = RobotModel.from_urdf_file(urdf)
model.load_geometry(loader)

# create robot artist
artist = RobotArtist(model)
artist

In [2]:
geo = artist.draw_visual()
viewer = ThreeJsViewer()
viewer.show(geometry=geo)

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "int32" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.…

<table style="font-size:100%">
<tr style="background: none;">
<td style="text-align: left"><pre><code class="python">from compas_fab.rhino import RobotArtist</code></pre></td>
<td style="text-align: left"><pre><code class="python">from compas_fab.blender import RobotArtist</code></pre></td>
</tr>
<tr>
<td><img src="images/robot_artist_rhino.jpg" /></td>
<td><img src="images/robot_artist_blender.jpg" /></td>
</tr>
</table>

## Joints

The joints are the elements in a robot which helps the links to travel in different kind of movements.
The three major types of joints are:

* Revolute: A hinge joint that rotates along the axis and has a limited range specified by the upper and lower limits.
* Prismatic: A sliding joint that slides along the axis, and has a limited range specified by the upper and lower limits.
* Fixed: Not really a joint because it cannot move, all degrees of freedom are locked.


In [8]:
from compas.robots import Joint

print(Joint.REVOLUTE)
print(Joint.PRISMATIC)
print(Joint.FIXED)

0
2
3


## Configuration

In [9]:
"""Example: create robot configuration
"""
from math import pi
from compas_fab.robots import Configuration

values = [1.5, pi]
types = [Joint.PRISMATIC, Joint.REVOLUTE]

config = Configuration(values, types)
config = Configuration.from_revolute_values([pi/2, 0., 0., pi/2, pi, 0])
config = Configuration.from_prismatic_and_revolute_values([8.312], [pi/2, 0., 0., 0., 2*pi, 0.8])


## Building your own robot

In [26]:
import math
from compas.geometry import Translation, Cylinder, Circle, Plane
from compas.robots import RobotModel, Joint
from compas_fab.robots import Configuration
from compas.datastructures import Mesh
from compas.geometry import Frame

from viewer import RobotArtist
from viewer import ThreeJsViewer

# create cylinder in yz plane
radius, length = 0.3, 5
cylinder = Cylinder(Circle(Plane([0, 0, 0], [1, 0, 0]), radius), length)
cylinder.transform(Translation([length / 2., 0, 0]))

# create robot
robot = RobotModel("robot", links=[], joints=[])

# add first link to robot
link0 = robot.add_link("world")

# add second link to robot
mesh = Mesh.from_shape(cylinder)
link1 = robot.add_link("link1", visual_mesh=mesh, visual_color=(0.2, 0.5, 0.6))

# add the joint between the links
axis = (0, 0, 1)
origin = Frame.worldXY()
robot.add_joint("joint1", Joint.CONTINUOUS, link0, link1, origin, axis)

# add another link
mesh = Mesh.from_shape(cylinder) # create a copy!
link2 = robot.add_link("link2", visual_mesh=mesh, visual_color=(0.5, 0.6, 0.2))

# add another joint to 'glue' the link to the chain
origin = Frame((length, 0, 0), (1, 0, 0), (0, 1, 0))
robot.add_joint("joint2", Joint.CONTINUOUS, link1, link2, origin, axis)

artist = RobotArtist(robot)
geo = artist.draw_visual()

In [27]:
viewer = ThreeJsViewer()
camera_position=[0.0, 10.0, 20.0]
viewer.show(camera_position=camera_position, geometry=geo)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.…

### Assignment Task 2

Build your own robot with a certain number of links and joints. Then create a `Configuration` with the correct joint types and values.
Then use the `RobotArtist` to `update` the robot with this configuration, so that it configures into the letter of your choice.
You can use the `RobotArtist` from `compas_fab.ghpython` or `compas_fab.rhino`.

<div align="center"><br><img src="images/assignment1_2.jpg" width="600" /></div>

## Forward and inverse kinematics

<img src="images/ik_fk.jpg" style="height: 500px" />

### Forward kinematics

The forward kinematics function calculates the pose of the robot’s end-effector from a configuration (configuration space to cartesian space).

The simplest way to calculate forward kinematics is based on the properties defined by the robot model:

In [18]:
from compas_fab.robots import Configuration
from compas_fab.robots.ur5 import Robot

robot = Robot(load_geometry=True)
configuration = Configuration.from_revolute_values([-2.238, -1.153, -2.174, 0.185, 0.667, 0.])

# Use forward kinematics of robot model
frame_RCF = robot.forward_kinematics(configuration, backend='model')
frame_WCF = robot.to_world_coords(frame_RCF)

print("Frame in the robot's coordinate system")
print(frame_RCF)
print("Frame in the world coordinate system")
print(frame_WCF)

Frame in the robot's coordinate system
Frame(Point(0.300, 0.100, 0.500), Vector(1.000, -0.000, -0.000), Vector(0.000, 1.000, -0.000))
Frame in the world coordinate system
Frame(Point(0.300, 0.100, 0.500), Vector(1.000, -0.000, -0.000), Vector(0.000, 1.000, -0.000))


In [19]:
from viewer import RobotArtist
from viewer import ThreeJsViewer

artist = RobotArtist(robot.model)
robot.artist = artist
robot.update(configuration)
geo = robot.draw_visual()
viewer = ThreeJsViewer()
camera_position=[1.0, 1.0, 1.0]
viewer.show(camera_position=camera_position, geometry=geo)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.…

## Question

For robotic arms, `inverse` is the tricky operation, do you know for which kind of robots `forward` is the tricky one?

## Backends : ROS

in google slides..

## Inverse kinematics

In [ ]:
from compas.geometry import Frame
from compas_fab.backends import RosClient
from compas_fab.robots import Configuration

frame = Frame([0.3, 0.1, 0.5], [1, 0, 0], [0, 1, 0])
start_configuration = Configuration.from_revolute_values([0] * 6)
group = "manipulator" # or robot.main_group_name

with RosClient() as client:
    robot.client = client
    config = robot.inverse_kinematics(frame, start_configuration, group)

print(config)

frame_RCF = robot.forward_kinematics(config, backend="model")
frame_WCF = robot.represent_frame_in_WCF(frame_RCF)
print(frame_WCF)